# Introduction to Semantic Kernel

Semantic Kernel is a lightweight, open-source development kit that lets you easily build AI agents and integrate the latest AI models

In [ ]:
pip install semantic-kernel

Create .env file and add following properties

GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY=""
AZURE_OPENAI_ENDPOINT=""
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="gpt-4o-mini"
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="gpt-4o-mini"
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="2023-03-15-preview"




In [11]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_call_behavior import FunctionCallBehavior

from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)


async def main():
    # Initialize the kernel
    kernel = Kernel()
    # Add Azure OpenAI chat completion
    kernel.add_service(
        AzureChatCompletion(
            service_id="default",
        )
    )

    # Enable planning
    execution_settings = AzureChatPromptExecutionSettings(tool_choice="auto")
    execution_settings.function_call_behavior = FunctionCallBehavior.EnableFunctions(
        auto_invoke=True, filters={}
    )

    # Create a history of the conversation
    history = ChatHistory()
    # Collect user input
    userInput = "Hello, how are you?"
    history.add_user_message(userInput)

    chat_function = kernel.add_function(
        prompt="""{{$chat_history}}{{$user_input}}""",
        function_name="chat",
        plugin_name="chat",
        prompt_execution_settings=execution_settings,
    )

    # Get the response from the AI
    result = kernel.invoke_stream(
        chat_function,
        user_input=userInput,
        chat_history=history,
    )

    # Print the results
    print("Assistant > " + await createSyncResponse(result))


async def createSyncResponse(answer) -> str:
    response = ""
    async for message in answer:
        response += str(message[0])
    return response


# Run the main function
if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError:
        # If already in an event loop, use await
        await main()

/tmp/ipykernel_109351/3501489966.py:26: DeprecationWarning: Use the `FunctionChoiceBehavior` `Auto` class method instead.
  execution_settings.function_call_behavior = FunctionCallBehavior.EnableFunctions(
The `function_call_behavior` parameter is deprecated. Please use the `function_choice_behavior` parameter instead.


Assistant > I'm just a program, but I'm here and ready to help you! How can I assist you today?


/tmp/ipykernel_109351/3501489966.py:67: RuntimeWarning: coroutine 'main' was never awaited
  await main()


Non-streaming chat completion
To use non-streaming chat completion, you can use the following code to generate a response from the AI agent.

In [23]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_call_behavior import FunctionCallBehavior

from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)


async def main():
    # Initialize the kernel
    kernel = Kernel()
    # Add Azure OpenAI chat completion
    chat_completion = AzureChatCompletion(
        service_id="default",
    )
    kernel.add_service(chat_completion)

    # Enable planning
    execution_settings = AzureChatPromptExecutionSettings()
    execution_settings.function_call_behavior = FunctionCallBehavior.EnableFunctions(
        auto_invoke=True, filters={}
    )

    chat_history = ChatHistory()
    chat_history.add_user_message("Hello, how are you?")

    response = await chat_completion.get_chat_message_content(
        chat_history=chat_history,
        kernel=kernel,
        settings=execution_settings,
    )

    # Print the results
    print("Assistant > " + response.content)


# Run the main function
if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError:
        # If already in an event loop, use await
        await main()

/tmp/ipykernel_109351/1012246501.py:25: DeprecationWarning: Use the `FunctionChoiceBehavior` `Auto` class method instead.
  execution_settings.function_call_behavior = FunctionCallBehavior.EnableFunctions(
The `function_call_behavior` parameter is deprecated. Please use the `function_choice_behavior` parameter instead.


Assistant > Hello! I'm here and ready to help. How can I assist you today?


/tmp/ipykernel_109351/1012246501.py:48: RuntimeWarning: coroutine 'main' was never awaited
  await main()
